In [ ]:
%AddJar https://jdbc.postgresql.org/download/postgresql-42.2.19.jar

In [ ]:
import org.apache.spark.sql.functions._

In [ ]:
val postgreDF = spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://10.0.0.5:5432/labdata")
    .option("dbtable", "domain_cats")
    .option("user", "andrey_blednykh2")
    .option("password", "AkRjOhAK")
    .option("driver", "org.postgresql.Driver")
    .load()

In [ ]:
val catSite = postgreDF
    .select("domain", "category")
    .withColumn("fullcat", concat(lit("shop_"), col("category")))

In [ ]:
catSite.show(2, 200, true)

In [ ]:
val hdfsDF = spark.read.json("/labs/laba03/weblogs.json")

In [ ]:
val logs = hdfsDF
    .select($"uid", explode($"visits").alias("struct_data"))
    .select($"uid", $"struct_data.timestamp".as("timestamp"), $"struct_data.url".as("url"))
    .withColumn("trick1", regexp_replace(col("url"), "https://", "http://"))
    .withColumn("trick2", regexp_replace(col("trick1"), "http://http://", "http://"))
    .withColumn("webhost",
        regexp_extract(col("trick2"),"^(([^:\\/?#]+):)?(\\/\\/([^\\/?#]*))?([^?#]*)(\\?([^#]*))?(#(.*))?", 4))
    .withColumn("domain", regexp_replace(col("webhost"), "^www.", ""))

In [ ]:
logs.show(2, 200, true)

In [ ]:
val visits = logs
    .join(catSite, Seq("domain"), "inner")
    .select("uid", "fullcat")
    .groupBy("uid")
    .pivot("fullcat")
    .agg(count("fullcat"))

In [ ]:
visits.show(2, 200, true)

In [ ]:
%AddJar https://repo1.maven.org/maven2/com/datastax/spark/spark-cassandra-connector_2.11/2.4.3/spark-cassandra-connector_2.11-2.4.3.jar

In [ ]:
val spark = SparkSession.builder()
    .config("spark.cassandra.connection.host", "10.0.0.5")
    .config("spark.cassandra.connection.port", "9042")
    .config("spark.cassandra.output.consistency.level", "ANY")
    .config("spark.cassandra.input.consistency.level", "ONE")      
    .getOrCreate()

In [ ]:
spark.stop()